In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
from cut_cross_entropy.transformers import cce_patch
from cut_cross_entropy import linear_cross_entropy
import torch
import transformers
import numpy as np
import random
from torchviz import make_dot

In [3]:
tokenizer = AutoTokenizer.from_pretrained('unsloth/Llama-3.2-3B-Instruct')

In [4]:
class Llama(LlamaForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        
    def forward(self, **kwargs):
        labels = kwargs.pop('labels')
        kwargs.pop('output_hidden_states', None)
        super_out = super().forward(**kwargs, output_hidden_states = True)
        if labels is not None:
            embeddings = super_out.hidden_states[-1]
            auto_shift_loss = linear_cross_entropy(embeddings, self.lm_head, labels, shift=True, impl = 'torch_compile')
            return {'loss': auto_shift_loss}
        return super_out

In [5]:
model = Llama.from_pretrained(
    'unsloth/Llama-3.2-3B-Instruct',
    torch_dtype = torch.bfloat16
).cuda()

In [6]:
model_auto = AutoModelForCausalLM.from_pretrained(
    'unsloth/Llama-3.2-3B-Instruct',
    torch_dtype = torch.bfloat16
).cuda()

In [7]:
rank = 256
peft_config = LoraConfig(
        lora_alpha=rank * 2,
        lora_dropout=0.0,
        r=rank,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["embed_tokens", "lm_head"],
    )

In [8]:
model = get_peft_model(model, peft_config)
model_auto = get_peft_model(model_auto, peft_config)

/home/husein/.local/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [9]:
input_ids = tokenizer.apply_chat_template([
    {'role': 'user', 'content': 'Hi!'}
], return_tensors = 'pt').cuda()
input_ids

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   2437,   4448,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  13347,      0, 128009]], device='cuda:0')

In [13]:
o = model(input_ids = input_ids, labels = input_ids)
o

{'loss': tensor(6.8273, device='cuda:0', grad_fn=<CompiledFunctionBackward>)}

In [14]:
o = model_auto(input_ids = input_ids, labels = input_ids)
o.loss

tensor(6.8273, device='cuda:0', grad_fn=<NllLossBackward0>)